## Playground for trying stuff here

In [1]:
import numpy as np

In [4]:
np.random.random(50)<0.5

array([False, False, False, False,  True, False, False, False, False,
        True, False,  True,  True, False, False,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True, False,
        True, False,  True, False, False, False,  True,  True,  True,
       False,  True,  True, False,  True,  True, False,  True, False,
        True,  True, False, False, False], dtype=bool)

In [29]:
a=np.array([[0,1],[2,3]])
a

array([[0, 1],
       [2, 3]])

In [34]:
a=tuple()
a

()

In [44]:
a = [1,2]

In [45]:
if a:
    print "here"

here
